##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analizador del modelo de TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/guide/model_analyzer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

La API del analizador de modelos TensorFlow Lite le ayuda a analizar modelos en formato TensorFlow Lite listando la estructura de un modelo.


## API del Analyzer del modelo

La siguiente API está disponible para el Analyzer del modelo de TensorFlow Lite.

```
tf.lite.experimental.Analyzer.analyze(model_path=None,
                                      model_content=None,
                                      gpu_compatibility=False)
```

Puede encontrar los detalles de la API en https://www.tensorflow.org/api_docs/python/tf/lite/experimental/Analyzer o ejecutar `help(tf.lite.experimental.Analyzer.analyze)` desde una terminal Python.


## Uso básico con un modelo Keras sencillo

El siguiente código muestra el uso básico del Analyzer del Modelo. Muestra el contenido del modelo Keras convertido en contenido de modelo TFLite, formateado como un objeto flatbuffer.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)


## Uso básico con el modelo Keras MobileNetV3Large

Esta API funciona con modelos de gran tamaño como MobileNetV3Large. Dado que la salida es grande, es posible que desee examinarla con su editor de texto favorito.

In [ ]:
model = tf.keras.applications.MobileNetV3Large()
fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)

## Comprobar la compatibilidad de la GPU

La API de ModelAnalyzer brinda una forma de comprobar la compatibilidad [del delegado de GPU](https://www.tensorflow.org/lite/performance/gpu) del modelo dado mediante la opción `gpu_compatibility=True`.


### Caso 1: Cuando el modelo es incompatible

El siguiente código muestra una forma de usar la opción `gpu_compatibility=True` para una simple función tf.function que usa `tf.slice` con un tensor 2D y `tf.cosh` que no son compatibles con el delegado de GPU.

Verá una `GPU COMPATIBILITY WARNING` por cada nodo que tenga problemas de compatibilidad.

In [ ]:
import tensorflow as tf

@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  return tf.cosh(x) + tf.slice(x, [1, 1], [1, 1])

converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [func.get_concrete_function()], func)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
fb_model = converter.convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

### Caso 1: Cuando el modelo es compatible

En este ejemplo, el modelo dado es compatible con el delegado de GPU.

**Nota:** Aunque la herramienta no encuentra ningún problema de compatibilidad, no garantiza que su modelo funcione bien con el delegado de GPU en todos los dispositivos. Podría producirse alguna incompatibilidad en runtime, como la falta de la función `CL_DEVICE_IMAGE_SUPPORT` por parte del backend OpenGL objetivo.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)